In [8]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,98676.093750,47114953674
2024-12-25,98675.914062,99478.750000,97593.468750,99299.195312,99299.195312,33700394629


In [9]:
import pandas as pd
import talib

data['ema20'] = talib.EMA(data['Close'], timeperiod=20)
data['ema50'] = talib.EMA(data['Close'], timeperiod=50)
data['rsi10'] = talib.RSI(data['Close'], timeperiod=10)
data['rsi30'] = talib.RSI(data['Close'], timeperiod=30)
data['rsi200'] = talib.RSI(data['Close'], timeperiod=200)

data['macd'], data['signal'], data['hist'] = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)
data.dropna(inplace=True)
data.to_csv('BTC-I.csv')
data

,Open,High,Low,Close,Adj Close,Volume,ema20,ema50,rsi10,rsi30,rsi200,macd,signal,hist,Price-Up
Date,,,,,,,,,,,,,,,
2015-04-05,253.761002,260.674988,251.942001,260.597992,260.597992,19649200,256.131664,258.804604,55.322540,50.121118,44.397749,-3.252124,-3.775657,0.523534,0
2015-04-06,260.721008,261.798004,254.574997,255.492004,255.492004,20034200,256.070744,258.674698,49.595801,48.750560,44.268371,-2.885118,-3.597550,0.712431,0
2015-04-07,255.274002,255.804993,252.205002,253.179993,253.179993,18467400,255.795434,258.459219,47.140702,48.134018,44.209743,-2.749134,-3.427866,0.678733,0
2015-04-08,253.063995,253.847000,244.214996,245.022003,245.022003,30086400,254.769393,257.932269,39.478769,46.010031,44.003080,-3.262043,-3.394702,0.132658,0
2015-04-09,244.751007,246.117996,239.399994,243.675995,243.675995,21643500,253.712879,257.373200,38.336478,45.666093,43.968998,-3.734095,-3.462580,-0.271515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919,98310.512205,92489.999087,37.870913,54.141525,56.212704,974.011455,2352.815347,-1378.803892,1
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,98676.093750,47114953674,98345.329495,92732.591034,50.713025,57.569224,56.908942,917.945789,2065.841435,-1147.895647,1
2024-12-25,98675.914062,99478.750000,97593.468750,99299.195312,99299.195312,33700394629,98436.173858,92990.104928,52.419615,58.075487,57.016216,913.264884,1835.326125,-922.061241,0


In [16]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target]


KeyError: "['sma', 'ema200'] not in index"

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle= False)


ValueError: With n_samples=0, test_size=0.01 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Stream_Crypto_Data 

In [5]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1m


In [6]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [7]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:25:56.034  95800.0  95873.5  95777.43  95873.49  3.58631
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:25:58.022  95800.0  95873.5  95777.43  95873.5  3.58869
                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:26:00.018  95800.0  95873.5  95777.43  95873.49  3.61952
                timestamp      Open      High       Low     Close  Volume
0 2024-12-26 18:26:00.018  95873.49  95873.49  95873.49  95873.49     0.0
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:26:02.025  95873.5  95873.5  95873.49  95873.5  0.00905
                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:26:04.017  95873.5  95873.5  95873.49  95873.49  0.07443
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:26:06.017  95873.5  95884.4  95873.49 

True